In [ ]:
import os
from stat import FILE_ATTRIBUTE_HIDDEN

import xxhash

## Define base path

In [ ]:
#BASE_PATH = r"\\192.168.1.13\tmp" 
BASE_PATH = r"D:\tmp\Neuer Ordner"

## Define functions

In [ ]:
def scan(dir, ext):
    subfolders, files = [], []

    for f in os.scandir(dir):
        if f.is_dir():
            if not bool(f.stat().st_file_attributes & FILE_ATTRIBUTE_HIDDEN):
                subfolders.append(f.path)
        if f.is_file():
            try:
                if (len(ext) == 0 or os.path.splitext(f.name)[1].lower() in ext) and os.path.getsize(f) <= 1024 * 1024 * 50: # 50 MB
                    files.append(f.path)
            except OSError:
                pass

    for dir in list(subfolders):
        f = scan(dir, ext)
        files.extend(f)
    return files

def hash_file(path):
    hasher = xxhash.xxh3_128()
    with open(path, 'rb') as f:
        hasher.update(f.read(4096))
        
    return hasher.hexdigest()
    

## Scan and hash all documents in a folder

In [ ]:
test_path = "O:\\Nextcloud"
files = scan(test_path, [".pdf", ".docx", ".txt", ".pptx"])
for f in files:
    hash_file(f)
    
len(files)

## Setup test "DB"

In [ ]:
mem_db : dict[str,str]  = {} # hash+path / path

## Get changes in directory

In [ ]:
files = scan(BASE_PATH, [])

added_paths = []

for f in files:
    file_hash = hash_file(f)
    
    path = mem_db.get(file_hash + f)
    
    if not path:
        added_paths.append(f) 
        mem_db[file_hash + f] = f
    
to_remove = list( set(mem_db.values()) - set(files))
swapped_db =  {val:key for key, val in mem_db.items()}

for f in to_remove:
    mem_db.pop(swapped_db[f])

if len(added_paths) > 0:
    print("Added paths:")
    for p in added_paths:
        print(f"\t {p}")

if len(to_remove) > 0:  
    print("Removed paths:")
    for p in to_remove:
        print(f"\t {p}")

if len(added_paths) > 0 or  len(to_remove) > 0:
    print()